In [8]:
""" ----- EDA -----
"""

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

""" ----- Feature Engineering -----
"""

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, Normalizer
# Log Transform
# Z-score

""" ----- Feature Selection -----
"""

from sklearn.feature_selection import VarianceThreshold, chi2, SelectKBest, f_regression, mutual_info_classif, RFE, SelectFromModel, r_regression

# Models for SelectFromModel 
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

""" ----- Model Selection -----
"""

from sklearn.linear_model import Ridge, Lasso, LinearRegression

# CV for models
from sklearn.linear_model import RidgeCV, LassoCV, LassoLarsCV
from sklearn.model_selection import train_test_split

import statsmodels.api as sm

from sklearn.pipeline import make_pipeline, Pipeline

In [9]:
df = pd.read_csv("data/AmesHousing.csv")

In [10]:
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [11]:
df = df.drop(["Order","PID","Alley","Fireplace Qu","Pool QC","Fence","Misc Feature"], axis=1)

In [12]:
df.shape

(2930, 75)

In [13]:
na_ratios = df.isnull().sum() / len(df)
na_ratios = na_ratios.drop(na_ratios[na_ratios == 0].index).sort_values(ascending = False)
na_ratios = pd.DataFrame({"Missing Proportion": na_ratios})
na_ratios = na_ratios.drop(na_ratios[na_ratios["Missing Proportion"] <0.005].index)
na_ratios

,Missing Proportion
Lot Frontage,0.167235
Garage Yr Blt,0.054266
Garage Qual,0.054266
Garage Finish,0.054266
Garage Cond,0.054266
Garage Type,0.053584
Bsmt Exposure,0.028328
BsmtFin Type 2,0.027645
Bsmt Qual,0.027304
Bsmt Cond,0.027304


In [16]:
df.dtypes

MS SubClass         int64
MS Zoning          object
Lot Frontage      float64
Lot Area            int64
Street             object
                   ...   
Mo Sold             int64
Yr Sold             int64
Sale Type          object
Sale Condition     object
SalePrice           int64
Length: 75, dtype: object

In [26]:
num_col = list(df.select_dtypes(include=['int64', 'float64']).columns)

In [29]:
# for col in num_col:
#     plt.figure()
#     plt.hist(x=df[col], bins=30)
#     plt.xlabel(col)
#     plt.show()